In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
import numpy as np
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt
import re
import nltk
import string
import csv

In [3]:
# download dataset from GDrive
downloaded = drive.CreateFile({'id':'1fzTqGXDtU9lmMsYQP5cc4S562iLfp9Uh'}) # use dataset's id from GDrive
downloaded.GetContentFile('dataset.csv')

# Read file as panda dataframe
df = pd.read_csv('dataset.csv')
df

,1,2,Label
0,NaN,tidy_tweet,NaN
1,0.0,Peneliti: Presiden perlu bentuk Badan Otorita ...,0.0
2,1.0,Pembangunan PLTN di Kalbar mantapkan pemindaha...,1.0
3,2.0,Prabowo setuju karena pemindahan ibu kota suda...,1.0
4,3.0,Pemindahan ibu kota bukti Jokowi ingin adanya ...,1.0
...,...,...,...
2898,2897.0,Semoga hal serupa juga dilakukan saat proses p...,1.0
2899,2898.0,Tolak Pemindahan Ibu Kota Tolak Kenaikan Iuran...,-1.0
2900,2899.0,Pemindahan ibu kota berhubungan dengan CINA Ke...,0.0
2901,2900.0,"Survei Median: 45,3% Publik Tidak Setuju Pemin...",-1.0


In [4]:
#drop index column 1 and row 0
df = df.drop("1", axis=1)
df = df.drop([0], axis=0)

In [5]:
# change the column names
df = df.rename(columns={"2":"tweet"})

In [6]:
# change the column names
df = df.rename(columns={"Label":"label"})
df

,tweet,label
1,Peneliti: Presiden perlu bentuk Badan Otorita ...,0.0
2,Pembangunan PLTN di Kalbar mantapkan pemindaha...,1.0
3,Prabowo setuju karena pemindahan ibu kota suda...,1.0
4,Pemindahan ibu kota bukti Jokowi ingin adanya ...,1.0
5,Lah itu kan sdh mau di ringankan dg pemindahan...,1.0
...,...,...
2898,Semoga hal serupa juga dilakukan saat proses p...,1.0
2899,Tolak Pemindahan Ibu Kota Tolak Kenaikan Iuran...,-1.0
2900,Pemindahan ibu kota berhubungan dengan CINA Ke...,0.0
2901,"Survei Median: 45,3% Publik Tidak Setuju Pemin...",-1.0


In [7]:
print(df.groupby('label').size())

label
-1.0     988
 0.0     865
 1.0    1042
dtype: int64


In [8]:
# output True if row above is the same
df.tweet.duplicated()

1       False
2       False
3       False
4       False
5       False
        ...  
2898    False
2899    False
2900    False
2901    False
2902    False
Name: tweet, Length: 2902, dtype: bool

In [9]:
df.tweet.duplicated().sum()

220

In [10]:
dfdroptest = df.drop_duplicates(keep = 'first')
dfdroptest

,tweet,label
1,Peneliti: Presiden perlu bentuk Badan Otorita ...,0.0
2,Pembangunan PLTN di Kalbar mantapkan pemindaha...,1.0
3,Prabowo setuju karena pemindahan ibu kota suda...,1.0
4,Pemindahan ibu kota bukti Jokowi ingin adanya ...,1.0
5,Lah itu kan sdh mau di ringankan dg pemindahan...,1.0
...,...,...
2898,Semoga hal serupa juga dilakukan saat proses p...,1.0
2899,Tolak Pemindahan Ibu Kota Tolak Kenaikan Iuran...,-1.0
2900,Pemindahan ibu kota berhubungan dengan CINA Ke...,0.0
2901,"Survei Median: 45,3% Publik Tidak Setuju Pemin...",-1.0


In [11]:
len(dfdroptest.tweet.unique())

2682

In [14]:
df.drop_duplicates(subset = ['tweet'], inplace = True, keep = 'first')

In [17]:
df.drop_duplicates()

,tweet,label
1,Peneliti: Presiden perlu bentuk Badan Otorita ...,0.0
2,Pembangunan PLTN di Kalbar mantapkan pemindaha...,1.0
3,Prabowo setuju karena pemindahan ibu kota suda...,1.0
4,Pemindahan ibu kota bukti Jokowi ingin adanya ...,1.0
5,Lah itu kan sdh mau di ringankan dg pemindahan...,1.0
...,...,...
2898,Semoga hal serupa juga dilakukan saat proses p...,1.0
2899,Tolak Pemindahan Ibu Kota Tolak Kenaikan Iuran...,-1.0
2900,Pemindahan ibu kota berhubungan dengan CINA Ke...,0.0
2901,"Survei Median: 45,3% Publik Tidak Setuju Pemin...",-1.0


In [18]:
df

,tweet,label
1,Peneliti: Presiden perlu bentuk Badan Otorita ...,0.0
2,Pembangunan PLTN di Kalbar mantapkan pemindaha...,1.0
3,Prabowo setuju karena pemindahan ibu kota suda...,1.0
4,Pemindahan ibu kota bukti Jokowi ingin adanya ...,1.0
5,Lah itu kan sdh mau di ringankan dg pemindahan...,1.0
...,...,...
2898,Semoga hal serupa juga dilakukan saat proses p...,1.0
2899,Tolak Pemindahan Ibu Kota Tolak Kenaikan Iuran...,-1.0
2900,Pemindahan ibu kota berhubungan dengan CINA Ke...,0.0
2901,"Survei Median: 45,3% Publik Tidak Setuju Pemin...",-1.0


In [19]:
dfdrop = df

In [20]:
dfdrop

,tweet,label
1,Peneliti: Presiden perlu bentuk Badan Otorita ...,0.0
2,Pembangunan PLTN di Kalbar mantapkan pemindaha...,1.0
3,Prabowo setuju karena pemindahan ibu kota suda...,1.0
4,Pemindahan ibu kota bukti Jokowi ingin adanya ...,1.0
5,Lah itu kan sdh mau di ringankan dg pemindahan...,1.0
...,...,...
2898,Semoga hal serupa juga dilakukan saat proses p...,1.0
2899,Tolak Pemindahan Ibu Kota Tolak Kenaikan Iuran...,-1.0
2900,Pemindahan ibu kota berhubungan dengan CINA Ke...,0.0
2901,"Survei Median: 45,3% Publik Tidak Setuju Pemin...",-1.0


In [21]:
len(dfdrop.tweet.unique())

2682

In [22]:
dfdrop[dfdrop['tweet'].isnull() == True]

,tweet,label


In [23]:
dfdrop[dfdrop['label'].isnull() == True]

,tweet,label
436,Beban berat bkn pada Jakarta sj. Pemindahan bu...,NaN
437,Pemindahan Ibu Kota membutuhkan dana Rp 466 Tr...,NaN
438,menabrak UNDANG-UNDANG REPUBLIK INDONESIA NO...,NaN
442,"Pada 2022 hingga 2024 mendatang, pemindahan Ib...",NaN
443,"Di Balik Pro Dan Kontra pemindahan Ibu Kota, P...",NaN
2771,"Minta Pemindahan Ibu Kota Dibatalkan, Amien Ra...",NaN


In [24]:
from google.colab import files

In [ ]:
dfdrop.to_csv('dfnodupli.csv', index = False)
files.download("dfnodupli.csv")